2021-05-06 [ADD] 여섯번째 분석 (이해강) - 일인당 연매출액 연평균 성장률

In [31]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# matplotlib 한글 폰트 오류 해결
import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='appleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system.... sorry.....')

In [32]:
df = pd.read_csv('company_nps_data.csv',encoding='ansi')
df

,회사ID,연매출액,년도,월,월별_연금보험료,월별_직원수
0,233757,41688077,2018,7,8912380,36
1,123464,1760695,2018,3,942380,5
2,342124,3221341,2018,7,781180,4
3,386420,4815584,2015,11,3795900,14
4,129243,9799736,2018,10,40724680,151
...,...,...,...,...,...,...
99796,171439,4630622,2017,2,2402740,11
99797,239388,1045021,2018,10,1428800,9
99798,272304,181898,2018,3,392560,4
99799,362194,2100218,2018,12,874780,3


In [60]:
# 일인당 매출액 컬럼 생성
df['일인당 매출액'] = round(df['연매출액']/df['월별_직원수'], 2)

# 회사ID와 년도별로 묶어 일인당 매출액의 연평균을 구하기
ndf = df.groupby(['회사ID','년도']).mean()

# 필요한 칼럼들만 추출
ndf.drop(['연매출액','월','월별_연금보험료','월별_직원수'],axis=1,inplace=True)

# 시계열로 나타낼 수 있게 피벗테이블 생성
ndf1 = ndf.pivot_table('일인당 매출액','회사ID','년도')
ndf1.index = ndf1.index.map(lambda x : str(x))
ndf1.columns = ndf1.columns.map(lambda x : str(x))

# '기간'이라는 칼럼 생성 후 임의의 값 부여
ndf1['기간'] = 0

# 한 행씩 접근하며 널값이 아닌 데이터들의 개수를 세서 '기간' 칼럼에 저장
for i in range(len(ndf1)):
    ndf1['기간'][i] = ndf1.iloc[i].notnull().sum()
    
# 그 후 2를 빼줌( "notnull()로 세어진'기간'칼럼" 과 "두 연도의 차이" )    
ndf1['기간']=ndf1['기간']-2

# '기간' 칼럼 제외한 새로운 데이터프레임으로 복사 (연도의 널값을 채워주기 위해) 
ndf2 = ndf1[['2015','2016','2017','2018','2019']]

# 계산의 편의상 시작값(2015로)과 종료값(2019로)을 맞춰주기 위해 널값 채우기
ndf2.T.fillna(method = 'ffill',inplace=True)
ndf2.T.fillna(method = 'bfill',inplace=True)

# 널값 채운 후 다시 '기간'칼럼 붙여주기
ndf2['기간'] = ndf1['기간']
ndf2

년도,2015,2016,2017,2018,2019,기간
회사ID,,,,,,
63,1.124874e+07,1.124874e+07,1.124874e+07,1.124874e+07,1.124874e+07,0
95,1.214752e+06,1.214752e+06,1.214752e+06,1.214752e+06,1.214752e+06,0
102,9.882968e+05,9.882968e+05,9.882968e+05,9.882968e+05,9.882968e+05,0
295,3.256360e+05,3.256360e+05,3.256360e+05,3.256360e+05,3.256360e+05,0
414,7.970422e+04,7.970422e+04,7.970422e+04,7.970422e+04,7.970422e+04,0
...,...,...,...,...,...,...
2400284,2.965137e+05,2.965137e+05,2.965137e+05,2.965137e+05,2.965137e+05,0
2437574,7.133867e+04,7.133867e+04,7.133867e+04,7.133867e+04,7.133867e+04,0
2693684,7.935867e+04,7.935867e+04,7.935867e+04,7.935867e+04,7.935867e+04,0


In [61]:
# 기간이 0인 데이터 소거
ndf2 = ndf2[ndf2['기간'] != 0]

# 일인당 매출액의 '연평균 성장률' 컬럼 생성
ndf2['연평균 성장률(%)'] = round(((ndf2['2019']/ndf2['2015'])**(1/ndf2['기간'])-1)*100, 2)

# 유니콘 데이터 기준 '연평균 성장률' ''-44.66% 미만' 데이터 소거
ndf2 = ndf2[ndf2['연평균 성장률(%)'] >= -44.66]

# 유니콘 데이터 기준 '일인당 연매출' '3604.83 미만' 데이터 소거
ndf2 = ndf2.loc[(ndf2['2015'] >= 3604.83) & (ndf2['2016'] >= 3604.83) & (ndf2['2017'] >= 3604.83) & (ndf2['2018'] >= 3604.83) & (ndf2['2019'] >= 3604.83)]

# 일인당 매출액의 '작년 대비 성장률' 컬럼 4개 생성
ndf2['15-16년 성장률'] = round(((ndf2['2016']-ndf2['2015'])/ndf2['2015'])*100, 1)
ndf2['16-17년 성장률'] = round(((ndf2['2017']-ndf2['2016'])/ndf2['2016'])*100, 1)
ndf2['17-18년 성장률'] = round(((ndf2['2018']-ndf2['2017'])/ndf2['2017'])*100, 1)
ndf2['18-19년 성장률'] = round(((ndf2['2019']-ndf2['2018'])/ndf2['2018'])*100, 1)

# 일인당 매출액의 '성장률 최고치' 컬럼 생성
ndf2['성장률 최고치']=0.0
for i in range(len(ndf2)):
    ndf2['성장률 최고치'][i]=max(ndf2['15-16년 성장률'][i],ndf2['16-17년 성장률'][i],ndf2['17-18년 성장률'][i],ndf2['18-19년 성장률'][i])

# 유니콘 데이터 기준 '성장률 최고치'가 23.2% 미만인 데이터 소거
ndf2 = ndf2.loc[ndf2['성장률 최고치']>=23.2]

ndf2

년도,2015,2016,2017,2018,2019,기간,연평균 성장률(%),15-16년 성장률,16-17년 성장률,17-18년 성장률,18-19년 성장률,성장률 최고치
회사ID,,,,,,,,,,,,
14271,94505.665000,290288.353333,300287.616667,419656.646667,404109.084167,4,43.80,207.2,3.4,39.8,-3.7,207.2
44851,144892.372500,144892.372500,144892.372500,297486.500000,297486.500000,1,105.32,0.0,0.0,105.3,0.0,105.3
122968,425174.740000,353555.739167,464383.680833,474796.361667,474796.361667,3,3.75,-16.8,31.3,2.2,0.0,31.3
123219,241154.307500,241154.307500,223040.454167,285657.455833,285657.455833,2,8.84,0.0,-7.5,28.1,0.0,28.1
123464,175333.591667,175333.591667,240461.263333,275457.940833,275457.940833,2,25.34,0.0,37.1,14.6,0.0,37.1
...,...,...,...,...,...,...,...,...,...,...,...,...
512507,92778.950000,92778.950000,92778.950000,131829.905833,131829.905833,1,42.09,0.0,0.0,42.1,0.0,42.1
514794,139716.491667,139716.491667,197967.075833,197967.075833,197967.075833,1,41.69,0.0,41.7,0.0,0.0,41.7
514934,64052.170000,64052.170000,79207.501667,83046.140833,83046.140833,2,13.87,0.0,23.7,4.8,0.0,23.7


### 최종 데이터가 5000개에서 1151개로 줄어든 것을 알 수 있다.

In [50]:
# 직원수가 0인 컬럼이 있어서, 연매출액을 나눠준뒤 inf값이 생김. 그것을 처리
ndf2 = ndf2.replace(np.inf,np.nan).dropna(axis=0)

### 일인당 연매출액 연평균 성장률 TOP50 추출

In [53]:
result = ndf2['연평균 성장률(%)'].sort_values(ascending=False).head(50)
result.to_excel('일인당 매출액 연평균 성장률.xlsx',encoding='ansi')